In [ ]:
# Data loading (features)

import pandas as pd
import json

json_features_file = open("flickr-spring-features.json")

parsed_json_features = json.load(json_features_file)

df_scatterplot = pd.json_normalize(parsed_json_features["features"])
# df_scatterplot = df_scatterplot[["latLng", "resolutions.ALL.fnRank", "resolutions.ALL.sigRank"]]
df_scatterplot = df_scatterplot[["resolutions.ALL.fnRank", "resolutions.ALL.sigRank"]]
df_scatterplot = df_scatterplot.rename(columns={"resolutions.ALL.fnRank": "fnRank", "resolutions.ALL.sigRank": "sigRank"})

df_scatterplot = df_scatterplot.dropna(subset=["fnRank"])
df_scatterplot = df_scatterplot.dropna(subset=["sigRank"])

return df_scatterplot


     fnRank  sigRank
0  1.000000      1.0
1  0.849920      1.0
2  0.640006      1.0
3  0.450896      1.0
4  0.373821      1.0


In [ ]:
# Vega-Lite

{ 
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json", 
  "params": [ {"name": "clickSelect", "select": "interval"} ], 
  "mark": { "type": "point", "cursor": "pointer" }, 
  "encoding": { 
    "x": {"field": "sigRank", "type": "quantitative"},
    "y": {"field": "fnRank", "type": "quantitative"}, 
    "fillOpacity": { 
        "condition": {"param": "clickSelect", "value": 1}, 
        "value": 0.3 
    }, 
    "color": { 
      "field": "interacted", 
      "type": "nominal", 
      "condition": {
        "test": "datum.interacted === '1'", "value": "red", "else": "blue"} } 
  }, 
  "config": { "scale": { "bandPaddingInner": 0.2 } } 
} 

